In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import sys
import numpy as np
import time
from operator import methodcaller
import requests
import datetime
import yaml
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
api_keys = None
with open("api_keys.yaml","r") as file_object:
    api_keys = yaml.load(file_object,Loader=yaml.SafeLoader)
api_counter = 0

In [4]:
def get_api_key(): # each API key is rate limited on how many API calls can be made per minute; therefore, use rotation of keys to prevent throttling
  global api_counter
  api_counter = (api_counter+1)%len(api_keys)
  return api_keys[api_counter]
def get_prev_api_key():
  counter = (api_counter+len(api_keys)-1)%len(api_keys)
  return api_keys[counter]


In [5]:
tsdaily_dfs, pe_ratio_dfs = {}, {}

In [6]:
# symbols = ['AMZN','PM','SBUX','DIS']
symbols = ['AAPL','AXP','GOOG','META','NVDA','AMZN','PM','SBUX','DIS','MSFT','NFLX','ICE']

In [7]:
def generate_dfs(symbol):
  print(symbol)
  # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
  function = 'EARNINGS'
  url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={get_api_key()}'
  r = requests.get(url)
  data = r.json()
  earnings_df = pd.DataFrame.from_records(data['quarterlyEarnings'])

  # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
  function = 'TIME_SERIES_DAILY'
  outputsize = 'full'
  url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={get_api_key()}&outputsize={outputsize}'
  r = requests.get(url)
  data = r.json()
  tsdaily_df = pd.DataFrame.from_records(data['Time Series (Daily)']).transpose()

  for col in tsdaily_df.columns:
    tsdaily_df[col] = pd.to_numeric(tsdaily_df[col])
  tsdaily_df.index = pd.to_datetime(tsdaily_df.index)

  for col in earnings_df.columns[2:]:
    earnings_df[col] = pd.to_numeric(earnings_df[col], errors='coerce')
  for col in earnings_df.columns[:2]:
    earnings_df[col] = pd.to_datetime(earnings_df[col], errors='coerce')

  earnings_df = earnings_df.set_index('fiscalDateEnding').sort_index()
  pe_ratio_df = pd.merge_asof(earnings_df, tsdaily_df[['4. close']], left_index=True, right_index=True, direction='nearest').sort_index(ascending=False)
  pe_ratio_df['PE Ratio'] = pe_ratio_df['4. close']/pe_ratio_df['reportedEPS']/4
  pe_ratio_df['PE Ratio (SMA)'] = pe_ratio_df['4. close']/pe_ratio_df['reportedEPS']/4
  pe_ratio_df.index = pd.to_datetime(pe_ratio_df.index)
  ctr = 0
  for idx, row in pe_ratio_df.iterrows():
    try:
      pe_ratio_df.iloc[ctr,7] = (pe_ratio_df.iloc[ctr,6]+pe_ratio_df.iloc[ctr+1,6]+pe_ratio_df.iloc[ctr+2,6]+pe_ratio_df.iloc[ctr+3,6])/4
    except:
      print(ctr)
      break
    ctr += 1
  pe_ratio_df = pe_ratio_df.sort_index()

  tsdaily_df = tsdaily_df.sort_index(ascending=False)
  tsdaily_df['TP'] = (tsdaily_df['2. high']+tsdaily_df['3. low']+tsdaily_df['4. close'])/3
  tsdaily_df['S.D.'] = tsdaily_df['TP']
  tsdaily_df['SMA'] = tsdaily_df['TP']
  ctr = 0
  for idx, row in tsdaily_df.iterrows():
    try:
      tsdaily_df.iloc[ctr,6] = tsdaily_df.iloc[ctr:ctr+20,5].std()
      tsdaily_df.iloc[ctr,7] = tsdaily_df.iloc[ctr:ctr+20,5].mean()
    except:
      print(ctr)
      break
    ctr += 1
  tsdaily_df = tsdaily_df.sort_index()
  tsdaily_df['BOLU'] = tsdaily_df['SMA'] + 2*tsdaily_df['S.D.']
  tsdaily_df['BOLD'] = tsdaily_df['SMA'] - 2*tsdaily_df['S.D.']
  tsdaily_df['state'] = 0
  for idx, row in tsdaily_df.iterrows():
    if row['4. close'] < row['BOLD']:
      tsdaily_df.loc[idx,'state'] = -1
    elif row['4. close'] > row['BOLU']:
      tsdaily_df.loc[idx,'state'] = 1

  return tsdaily_df, pe_ratio_df

In [8]:
for symbol in symbols:
  if symbol not in tsdaily_dfs.keys() or symbol not in pe_ratio_dfs.keys():
    try:
      tsdaily_dfs[symbol], pe_ratio_dfs[symbol] = generate_dfs(symbol)
    except KeyError:
      print(f'Failed at {get_prev_api_key()}')
      break

AAPL
108
AXP
108
GOOG
36
META
44
NVDA
95
AMZN
103
PM
60
SBUX
108
DIS
108
MSFT
108
NFLX
83
ICE
70


In [9]:
tsdaily_dfs.keys()

dict_keys(['AAPL', 'AXP', 'GOOG', 'META', 'NVDA', 'AMZN', 'PM', 'SBUX', 'DIS', 'MSFT', 'NFLX', 'ICE'])

In [10]:
# To CSVs
# for symbol in symbols:
for symbol in tsdaily_dfs.keys():
    tsdaily_dfs[symbol].to_csv(f'tsdaily_dfs/{symbol}.csv')
for symbol in pe_ratio_dfs.keys():
    pe_ratio_dfs[symbol].to_csv(f'pe_ratio_dfs/{symbol}.csv')

In [11]:
# Read CSVs
# for symbol in symbols:
for symbol in [i[:-4] for i in os.listdir('tsdaily_dfs')]:
    tsdaily_df = pd.read_csv(f'tsdaily_dfs/{symbol}.csv',index_col=0)
    tsdaily_df.index = pd.to_datetime(tsdaily_df.index)
    tsdaily_dfs[symbol] = tsdaily_df

for symbol in [i[:-4] for i in os.listdir('pe_ratio_dfs')]:
    pe_ratio_df = pd.read_csv(f'pe_ratio_dfs/{symbol}.csv',index_col=0)
    pe_ratio_df.index = pd.to_datetime(pe_ratio_df.index)
    for col in pe_ratio_df.columns[:1]:
        pe_ratio_df[col] = pd.to_datetime(pe_ratio_df[col], errors='coerce')
    pe_ratio_dfs[symbol] = pe_ratio_df

In [12]:
def create_buy_sell_pairs(symbol, start_date = '2019-01-01', end_date = datetime.date.today()):
  curr_state = 0
  buy_sell_pairs = []
  curr_buy = None
  for idx, row in tsdaily_dfs[symbol][start_date:].iterrows():
    if row['state'] == -1 and curr_state == 0:
      curr_buy = (idx,row['4. close'])
      curr_state = -1
    elif row['state'] == 1 and curr_state == -1:
      curr_sell = (idx,row['4. close'])
      buy_sell_pairs.append((curr_buy,curr_sell))
      curr_state = 0
  return buy_sell_pairs
def generate_bollinger_returns(buy_sell_pair):
  return sum([s[1] for b,s in buy_sell_pair]) - sum([b[1] for b,s in buy_sell_pair])

In [13]:
def plot_graph(symbol, start_date = '2021-01-01', end_date = datetime.date.today()):
  step = 30
  fig = make_subplots(specs=[[{"secondary_y": True}]])

  tsdaily_df = tsdaily_dfs[symbol].loc[start_date: end_date]
  # Add traces
  fig.add_trace(go.Scatter(x=tsdaily_df.index, y=tsdaily_df['4. close'], name="Daily Price"),secondary_y=False,)
  fig.add_trace(go.Scatter(x=tsdaily_df.index, y=tsdaily_df['SMA'], name="Mid"),secondary_y=False,)
  fig.add_trace(go.Scatter(x=tsdaily_df.index, y=tsdaily_df['BOLU'], name="Upper"),secondary_y=False,)
  fig.add_trace(go.Scatter(x=tsdaily_df.index, y=tsdaily_df['BOLU'], name="Lower"),secondary_y=False,)


  pe_ratio_df = pe_ratio_dfs[symbol].loc[start_date: end_date]
  # Add traces
  fig.add_trace(go.Scatter(x=pe_ratio_df.index, y=pe_ratio_df['PE Ratio'], name="PE"),secondary_y=True,)
  fig.add_trace(go.Scatter(x=pe_ratio_df.index, y=pe_ratio_df['PE Ratio (SMA)'], name="PE (SMA)"),secondary_y=True,)

  bsp = create_buy_sell_pairs(symbol,start_date,end_date)
  for b,s in bsp:
    fig.add_trace(go.Scatter(x=(b[0], s[0]), y=(b[1], s[1]), name="Daily Price",showlegend=False,line_color='#000000'),secondary_y=False)

  # Add figure title
  fig.update_layout(title_text=f"{symbol}: {generate_bollinger_returns(bsp):.2f}")

  # Set x-axis title
  fig.update_xaxes(title_text="Time")

  # Set y-axes titles
  fig.update_yaxes(title_text="<b>Stock Price</b>", secondary_y=False)
  fig.update_yaxes(title_text="<b>PE Ratio</b>", secondary_y=True)

  fig.show()

In [14]:
plot_graph('GOOG')

In [15]:
# calculate Bollinger squeeze manually using SMA, since the Bollinger band is off for few of the stocks (works fine only for AAPL and AXP)
# https://www.britannica.com/money/bollinger-bands-indicator
# https://www.investopedia.com/articles/technical/04/030304.asp

In [16]:
buy_sell_net= {}
for symbol in tsdaily_dfs.keys():
  buy_sell_net[symbol] = generate_bollinger_returns(create_buy_sell_pairs(symbol))

In [17]:
buy_sell_net

{'AAPL': 98.4699999999998,
 'AXP': 52.50000000000023,
 'GOOG': 795.1499999999942,
 'META': -24.96000000000049,
 'NVDA': -31.829999999999927,
 'AMZN': 1122.0,
 'PM': 1.5,
 'SBUX': 18.37999999999988,
 'DIS': -12.720000000000255,
 'MSFT': 148.51999999999998,
 'NFLX': 54.07000000000062,
 'ICE': 46.5900000000006}

In [18]:
for symbol in tsdaily_dfs.keys():
  plot_graph(symbol,start_date = '2023-01-01')